In [ ]:
!nvidia-smi

Thu Mar  6 13:51:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#!pip install accelerate
!pip install transformers==4.45.2
#!pip install bitsandbytes
!pip install datasets
!pip install rouge-score
!pip install pymorphy3
!pip install peft
#!pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_sco

In [ ]:
!git clone https://github.com/RefalMachine/llmtf_open
%cd llmtf_open
!wget https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/train.jsonl

Cloning into 'llmtf_open'...
remote: Enumerating objects: 639, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 639 (delta 150), reused 135 (delta 70), pack-reused 420 (from 1)
Receiving objects: 100% (639/639), 2.32 MiB | 7.97 MiB/s, done.
Resolving deltas: 100% (435/435), done.
/content/llmtf_open
--2025-03-06 13:53:47--  https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1291979 (1.2M) [text/plain]
Saving to: ‘train.jsonl’

train.jsonl         100%[===================>]   1.23M  --.-KB/s    in 0.04s   

2025-03-06 13:53:49 (35.1 MB/s) - ‘train.jsonl’ saved [1291979/1291979]



In [ ]:
!ls

conversation_configs  llm_as_a_judge_baselines		  run_evaluate_multinode_multigpu.sh
Dockerfile	      llmtf				  run_evaluate_singlenode_multigpu.sh
eval_grammar.py       README.md				  run_llm_as_a_judge.py
evaluate_model.py     requirements.txt			  todo.txt
examples	      run_evaluate_multinode_multigpu.py  train.jsonl


In [ ]:
import codecs
import json
import copy
from collections import OrderedDict, defaultdict
import numpy as np
from tqdm import tqdm
import os
from datasets import load_dataset, Dataset
from typing import Dict, List, Tuple
from llmtf.metrics import mean, metric_max_over_ground_truths, f1_macro_score
import transformers.data.metrics.squad_metrics as squad_metrics
import re
from llmtf.base import Task, SimpleFewShotHFTask, LLM
from difflib import SequenceMatcher
import pandas as pd
import string

class MultiQ(SimpleFewShotHFTask):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.method = 'generate'
        self.dataset_name = 'multiq'
        self._max_new_tokens = 64

    @classmethod
    def name(cls):
        return 'darumeru/MultiQ'

    def dataset_args(self) -> Dict:
        return {'path': 'RefalMachine/darumeru', 'name': self.dataset_name}

    def aggregation(self) -> Dict:
        return {"f1": mean, "em": mean}

    def evaluate(self, sample, y_pred) -> Dict:
        y_true = [answer["segment"] for answer in sample['outputs']]
        f1 = metric_max_over_ground_truths(squad_metrics.compute_f1, y_pred, y_true)
        em = metric_max_over_ground_truths(squad_metrics.compute_exact, y_pred, y_true)

        return {
            "f1": f1,
            "em": em,
        }
    def test_split_name(self) -> str:
        return 'test'

    def prompt_split_name(self) -> str:
        return 'prompt'

    def create_messages(self, sample, with_answer=None) -> List[Dict]:
        # ignoring with_answer because it's already taken into account in the darumeru dataset
        messages = sample['messages']
        inputs = sample['inputs']
        for m in messages:
            m['content'] = m['content'].format(**inputs)
        return messages

In [ ]:
task = MultiQ()

In [ ]:
from llmtf.model import HFModel

model = HFModel(attn_implementation='sdpa', device_map='cuda')
model.from_pretrained('RefalMachine/RuadaptQwen2.5-1.5B-instruct')

model.generation_config.max_new_tokens = 200
model.generation_config.repetition_penalty = 1.0
model.generation_config.do_sample = False
model.generation_config.temperature = 0.0
model.generation_config

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.34M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/2.28M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

INFO: 2025-03-06 13:57:15,779: llmtf.base.hfmodel: Set eos_token_id in generation_config to [145111]
INFO:llmtf.base.hfmodel:Set eos_token_id in generation_config to [145111]
INFO: 2025-03-06 13:57:15,781: llmtf.base.hfmodel: Model id: RefalMachine/RuadaptQwen2.5-1.5B-instruct
INFO:llmtf.base.hfmodel:Model id: RefalMachine/RuadaptQwen2.5-1.5B-instruct
INFO: 2025-03-06 13:57:15,783: llmtf.base.hfmodel: Leading space: False
INFO:llmtf.base.hfmodel:Leading space: False


GenerationConfig {
  "bos_token_id": 145109,
  "eos_token_id": [
    145111
  ],
  "max_length": 32768,
  "max_new_tokens": 200,
  "pad_token_id": 145109,
  "stop_strings": [
    "<|im_end|>"
  ],
  "temperature": 0.0,
  "top_k": 40,
  "top_p": 0.9,
  "trust_remote_code": false
}

In [ ]:
from llmtf.evaluator import Evaluator
evaluator = Evaluator()

evaluator.evaluate_dataset(
    task=task,
    model=model,
    output_dir='./multiq',
    max_len=4000,
    few_shot_count=0,
    generation_config=None, # will use model.generation_config by default
    batch_size=4,
    max_sample_per_dataset=200
)

INFO: 2025-03-06 13:57:20,024: llmtf.base.hfmodel: Updated generation_config.eos_token_id: [145111]
INFO:llmtf.base.hfmodel:Updated generation_config.eos_token_id: [145111]
INFO: 2025-03-06 13:57:20,025: llmtf.base.hfmodel: Updated generation_config.stop_strings: ['<|im_end|>']
INFO:llmtf.base.hfmodel:Updated generation_config.stop_strings: ['<|im_end|>']


README.md:   0%|          | 0.00/4.31k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/3.01M [00:00<?, ?B/s]

prompt.jsonl:   0%|          | 0.00/31.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1046 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/10 [00:00<?, ? examples/s]

100%|██████████| 200/200 [00:00<00:00, 567.10it/s]
INFO: 2025-03-06 13:57:22,869: llmtf.base.darumeru/MultiQ: Loading Dataset: 2.84s
INFO:llmtf.base.darumeru/MultiQ:Loading Dataset: 2.84s
  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `40` --

In [ ]:
!ls ./multiq/

darumeru_MultiQ.jsonl  darumeru_MultiQ_params.jsonl  darumeru_MultiQ_total.jsonl


In [ ]:
!cat ./multiq/darumeru_MultiQ_total.jsonl

{
    "task_name": "darumeru/MultiQ",
    "results": {
        "f1": 0.3217560871948909,
        "em": 0.23
    },
    "leaderboard_result": 0.27587804359744544
}


In [ ]:
!cat ./multiq/darumeru_MultiQ_params.jsonl

{
    "custom_generation_config": null,
    "model_params": {
        "model_name_or_path": "RefalMachine/RuadaptQwen2.5-1.5B-instruct",
        "generation_config": {
            "bos_token_id": 145109,
            "eos_token_id": [
                145111
            ],
            "max_length": 32768,
            "max_new_tokens": 200,
            "pad_token_id": 145109,
            "stop_strings": [
                "<|im_end|>"
            ],
            "temperature": 0.0,
            "top_k": 40,
            "top_p": 0.9,
            "transformers_version": "4.45.2",
            "trust_remote_code": false
        },
        "conversation_template": {
            "system_prompt": "",
            "system_message_template": "<|im_start|>system\n{content}<|im_end|>\n",
            "user_message_template": "<|im_start|>user\n{content}<|im_end|>\n",
            "bot_message_template": "<|im_start|>assistant\n{content}<|im_end|>\n",
            "bot_message_template_incomplete": "<|im_st